In [2]:
# Setup: imports and configuration
import os
import cv2
import numpy as np

from court_vision import detect_playable_area

# User: set your input video path here
VIDEO_PATH = os.path.abspath('raw_videos/Monica Greene unedited tennis match play.mp4')  # change this
OUTPUT_DIR = os.path.abspath('sanity_check_clips')
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_VIDEO = os.path.join(OUTPUT_DIR, 'court_detection_test.mp4')

print('Input video:', VIDEO_PATH)
print('Output video:', OUTPUT_VIDEO)


Input video: /Users/ismaelrobles-razzaq/Desktop/tennis_tracker/raw_videos/Monica Greene unedited tennis match play.mp4
Output video: /Users/ismaelrobles-razzaq/Desktop/tennis_tracker/sanity_check_clips/court_detection_test.mp4


In [3]:
# Detection and visualization over first 60 seconds
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError(f'Failed to open video: {VIDEO_PATH}')

fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
max_frames = int(60 * fps)

ret, frame0 = cap.read()
if not ret or frame0 is None:
    cap.release()
    raise RuntimeError('Could not read first frame for playable area detection')

playable_poly = detect_playable_area(frame0, margin_pixels=50)  # (4,2)

h, w = frame0.shape[:2]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (w, h))

# Reset to beginning
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

num_written = 0
alpha = 0.4  # overlay opacity
poly_color = (255, 200, 80)  # BGR light color
outline_color = (0, 220, 255)

while True:
    if num_written >= max_frames:
        break
    ret, frame = cap.read()
    if not ret or frame is None:
        break

    overlay = frame.copy()
    # Fill polygon on overlay
    pts = playable_poly.reshape((-1, 1, 2))
    cv2.fillPoly(overlay, [pts], color=poly_color)
    # Blend overlay
    blended = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)
    # Outline
    cv2.polylines(blended, [pts], isClosed=True, color=outline_color, thickness=3)

    writer.write(blended)
    num_written += 1


In [4]:
# Cleanup
cap.release()
writer.release()
print(f'Saved court detection clip to: {OUTPUT_VIDEO}')


Saved court detection clip to: /Users/ismaelrobles-razzaq/Desktop/tennis_tracker/sanity_check_clips/court_detection_test.mp4
